In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [5]:
dataset = tfds.load('imdb_reviews',as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Shuvo\tensorflow_datasets\imdb_reviews\plain_text\incomplete.84OY3E_1.0.0\imdb_reviews-trai…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Shuvo\tensorflow_datasets\imdb_reviews\plain_text\incomplete.84OY3E_1.0.0\imdb_reviews-test…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\Shuvo\tensorflow_datasets\imdb_reviews\plain_text\incomplete.84OY3E_1.0.0\imdb_reviews-unsu…

Dataset imdb_reviews downloaded and prepared to C:\Users\Shuvo\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_dataset,test_dataset=dataset['train'],dataset['test']
batch_size=32